# Agent Delegation

Delegation is when an agent invokes another agent through a tool while maintaining control of the overall task. The parent agent decides when to delegate, receives the result, and incorporates it into its final response.

In [ ]:
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext
from agentic_patterns.core.agents import get_agent, run_agent

## The Scenario

A research assistant helps users explore topics. When it makes factual claims, it can delegate verification to a fact-checker specialist. The research agent stays in control, deciding when verification is needed and how to incorporate the results.

## The Specialist Agent

The fact-checker is a focused agent that evaluates specific claims. It returns structured output indicating whether the claim is accurate and providing context.

In [ ]:
class FactCheckResult(BaseModel):
    claim: str = Field(description="The claim that was checked")
    verdict: str = Field(description="accurate, inaccurate, or partially accurate")
    explanation: str = Field(description="Brief explanation of the verdict")


fact_checker = get_agent(
    output_type=FactCheckResult,
    system_prompt="""You are a fact-checker. Evaluate claims for accuracy.
Be precise and cite your reasoning. Focus only on verifiable facts."""
)

## The Delegation Tool

The tool wraps the specialist agent. When the parent agent calls this tool, it runs the fact-checker and returns the result. The `RunContext` provides access to usage tracking.

In [ ]:
async def fact_check(ctx: RunContext[None], claim: str) -> str:
    """Verify a factual claim by delegating to a fact-checking specialist."""
    print(f"[Delegating to fact-checker] Claim: {claim}")
    
    agent_run, _ = await run_agent(
        fact_checker,
        f"Fact-check this claim: {claim}"
    )
    result = agent_run.result.output
    
    # Propagate usage from the delegated agent to the parent
    ctx.usage.incr(agent_run.result.usage())
    
    print(f"[Fact-checker result] {result.verdict}: {result.explanation}")
    return f"Verdict: {result.verdict}. {result.explanation}"

## The Parent Agent

The research assistant has access to the fact_check tool. It decides autonomously when to verify claims.

In [ ]:
research_agent = get_agent(
    tools=[fact_check],
    system_prompt="""You are a research assistant. Help users explore topics accurately.
When you make specific factual claims that could be verified, use the fact_check tool.
After fact-checking, incorporate the results into your response."""
)

## Running the Delegation

When the research agent encounters a claim worth verifying, it calls the fact_check tool. The specialist runs, and the result flows back to the parent.

In [ ]:
prompt = """Tell me about the speed of light and its discovery. 
Please verify the key facts you mention."""

agent_run, _ = await run_agent(research_agent, prompt, verbose=True)

print("\n" + "="*50)
print("FINAL RESPONSE:")
print("="*50)
print(agent_run.result.output)

## Unified Usage Tracking

Because we called `ctx.usage.incr()` in the tool, the parent agent's usage includes tokens from the delegated agent.

In [ ]:
usage = agent_run.result.usage()
print(f"Total tokens (including delegated calls): {usage.total_tokens}")
print(f"  Request tokens: {usage.request_tokens}")
print(f"  Response tokens: {usage.response_tokens}")